<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>

# Analyze Facebook Data Using IBM Watson and IBM Watson Studio

This is a three-part notebook written in `Python_3.5` meant to show how anyone can enrich and analyze a combined dataset of unstructured and structured information with IBM Watson and IBM Watson Studio. For this example we are using a standard Facebook Analytics export which features texts from posts, articles and thumbnails, along with standard performance metrics such as likes, shares, and impressions. 

**Part I** will use the Natual Language Understanding, Visual Recognition and Tone Analyzer Services from IBM Watson to enrich the Facebook Posts, Thumbnails, and Articles by pulling out `Emotion Tones`, `Social Tones`, `Language Tones`, `Entities`, `Keywords`, and `Document Sentiment`. The end result of Part I will be additional features and metrics we can test, analyze, and visualize in Part III. 

**Part II** will be used to set up the visualizations and tests we will run in Part III. The end result of Part II will be multiple pandas DataFrames that will contain the values, and metrics needed to find insights from the Part III tests and experiments.

**Part III** will include services from IBM's Watson Studio, including IBM's own data visualization library PixieDust. In Part III we will run analysis on the data from the Facebook Analytics export, such as the number of likes, comments, shares, to the overall reach for each post, and will compare it to the enriched data we pulled in Part I.


#### You should only need to change data in the Setup portion of this notebook. All places where you see  <span style="color: red"> User Input </span> is where you should be adding inputs. 

### Table of Contents

### [**Part I - Enrich**](#part1)<br>
1. [Setup](#setup)<br>
   1.1 [Install Watson Developer Cloud and BeautifulSoup Packages](#setup1)<br>
   1.2 [Install PixieDust](#pixie)<br> 
   1.3 [Restart Kernel](#restart)<br> 
   1.4 [Import Packages and Libraries](#setup2)<br>
   1.5 [Add Service Credentials From IBM Cloud for Watson Services](#setup3)<br>
2. [Load Data](#load)<br>
   2.1 [Load Data From Cloud Object Storage as a pandas DataFrame](#load1)<br>
   2.2 [Set Variables](#load2)<br>
3. [Prepare Data](#prepare)<br>
   3.1 [Data Cleansing with Python](#prepare1)<br>
   3.2 [Beautiful Soup to Extract Thumbnails and Extented Links](#prepare2)<br>
4. [Enrich Data](#enrich)<br>
   4.1 [NLU for Post Text](#nlupost)<br>
   4.2 [NLU for Thumbnail Text](#nlutn)<br>
   4.3 [NLU for Article Text](#nlulink)<br>
   4.4 [Tone Analyzer for Post Text](#tonepost)<br>
   4.5 [Tone Analyzer for Article Text](#tonearticle)<br>
   4.6 [Visual Recognition](#visual)<br>
5. [Write Data](#write)<br>
   5.1 [Convert DataFrame to new CSV](#write1)<br>
   5.2 [Write Data to Cloud Object Storage](#write2)<br>
    
### [**Part II - Data Preparation**](#part2)<br>
1. [Prepare Data](#prepare)<br>
   1.1 [Create Multiple DataFrames for Visualizations](#visualizations)<br>
   1.2 [Create a Consolidated Tone DataFrame](#tone)<br>
   1.3 [Create a Consolidated Keyword DataFrame](#keyword)<br>
   1.4 [Create a Consolidated Entity DataFrame](#entity)<br>
  
### [**Part III - Analyze**](#part3)<br>

1. [Setup](#2setup)<br> 
    1.1 [Assign Variables](#2setup2)<br>
2. [Visualize Data](#2visual)<br>
    2.1 [Run PixieDust Visualization Library with Display() API](#2visual1)
   
### Learn more about the technology used:

* [Natual Language Understanding](https://www.ibm.com/watson/developercloud/natural-language-understanding.html)
* [Tone Analyzer](https://www.ibm.com/watson/developercloud/tone-analyzer.html)
* [Visual Recognition](https://www.ibm.com/watson/services/visual-recognition/)
* [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [PixieDust](https://github.com/ibm-cds-labs/pixiedust) (Part III)

# <a id="part1"></a>Part I - Enrich

## <a id="setup"></a>1. Setup
To prepare your environment, you need to install some packages and enter credentials for the two Watson services.

### <a id="setup1"></a> 1.1 Install Latest Watson Developer Cloud and Beautiful Soup Packages
You need to install these packages:
 - [Watson Developer Cloud](https://github.com/watson-developer-cloud/python-sdk): a client library for Watson services.
 - <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target="_blank" rel="noopener noreferrer">Beautiful Soup</a>: a library to parse data from HTML for enriching the Facebook data.
 - <a href="https://ibm-cds-labs.github.io/pixiedust/" target="_blank" rel="noopener noreferrer">PixieDust</a>: a library to visualize the data. 

Install the Watson Developer Cloud package:

In [ ]:
!pip -q install watson-developer-cloud==2.1.0

Install the Beautiful Soup package:

In [ ]:
!pip -q install --upgrade beautifulsoup4

<a id="pixie"></a>
### 1.2 Install PixieDust Library
This notebook provides an overview of how to use the PixieDust library to analyze and visualize various data sets. If you are new to PixieDust or would like to learn more about the library, please go to this [Introductory Notebook](https://apsportal.ibm.com/exchange/public/entry/view/5b000ed5abda694232eb5be84c3dd7c1) or visit the [PixieDust Github](https://ibm-cds-labs.github.io/pixiedust/). The `Setup` section for this notebook uses instructions from the [Intro To PixieDust](https://github.com/ibm-cds-labs/pixiedust/blob/master/notebook/Intro%20to%20PixieDust.ipynb) notebook.

Ensure you are running the latest version of PixieDust by running the following cell. Do not run this cell if you installed PixieDust locally from source and want to continue to run PixieDust from source.

In [ ]:
!pip -q install --user --upgrade pixiedust

<a id="restart"></a>
### 1.3 Restart Kernel
> Required after installs/upgrades only.

If any libraries were just installed or upgraded, <span style="color: red">restart the kernel</span> before continuing. After this has been done once, you might want to comment out the `!pip install` lines above for cleaner output and a faster "Run All".

<a id="setup2"></a>
### 1.4 Import Packages and Libraries
> Tip: To check if you have a package installed, open a new cell and write: `help(<package-name>)`.

In [1]:
import json
import sys

import watson_developer_cloud
from watson_developer_cloud import ToneAnalyzerV3, VisualRecognitionV3
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, EmotionOptions, SentimentOptions

import operator
from functools import reduce
from io import StringIO
import numpy as np
from bs4 import BeautifulSoup as bs
from operator import itemgetter
from os.path import join, dirname
import pandas as pd
import numpy as np
import requests
import pixiedust

# Suppress some pandas warnings
pd.options.mode.chained_assignment = None  # default='warn'
# Suppress SSL warnings
requests.packages.urllib3.disable_warnings()

Pixiedust database opened successfully


<a id='setup3'></a>
### 1.5 Add Service Credentials From IBM Cloud for Watson Services
Edit the following cell to provide your credentials for Watson Visual Recognition, Natural Language Understanding and Tone Analyzer.

You must create a Watson Natural Language Understanding service, a Watson Tone Analyzer service, and a Watson Visual Recognition service on [IBM Cloud](bluemix.net).

1. Create a service for [Natural Language Understanding (NLU)](https://www.ibm.com/watson/developercloud/natural-language-understanding.html). 
1. Create a service for [Tone Analyzer](https://console.ng.bluemix.net/catalog/services/tone-analyzer?taxonomyNavigation=watson).
1. Create a service for [Visual Recognition](https://console.ng.bluemix.net/catalog/services/visual-recognition).
1. Insert the username and password values for your NLU and Tone Analyzer services and the API key for your Visual Recognition service in the following cell. Do not change the values of the `version` fields.
1. Run the cell.

###  <span style="color: red"> _User Input_</span> 

In [2]:
# @hidden_cell

# Watson Visual Recognition
# VISUAL_RECOGNITION_API_KEY = '<add_vr_api_key>'

# Watson Natural Launguage Understanding (NLU)
NATURAL_LANGUAGE_UNDERSTANDING_USERNAME = 'f1e5dce1-3e78-43ba-bbcc-4c5ca3eefbf5'
NATURAL_LANGUAGE_UNDERSTANDING_PASSWORD = 'mgwU4ZEaUPYY'

# Watson Tone Analyzer
TONE_ANALYZER_USERNAME = '<add_tone_analyzer_username>'
TONE_ANALYZER_PASSWORD = '<add_tone_analyzer_password>'

In [3]:
# Create the Watson clients

nlu = watson_developer_cloud.NaturalLanguageUnderstandingV1(version='2017-02-27',
                                                            username=NATURAL_LANGUAGE_UNDERSTANDING_USERNAME,
                                                            password=NATURAL_LANGUAGE_UNDERSTANDING_PASSWORD)
#tone_analyzer = ToneAnalyzerV3(version='2016-05-19',
#                                username=TONE_ANALYZER_USERNAME,
#                                password=TONE_ANALYZER_PASSWORD)

# visual_recognition = VisualRecognitionV3(version='2018-05-22', iam_apikey=VISUAL_RECOGNITION_API_KEY)


## <a id='load'></a>2. Load Data
The data you'll analyzing is a sample of a standard export of the Facebook Insights Post information from the <a href="https://www.facebook.com/ibmwatson/" target="_blank" rel="noopener noreferrer">IBM Watson Facebook page</a>. Engagement metrics such as clicks, impressions, and so on, are altered and do not reflect actual post performance data. The data is on the Watson Studio community page.

### 2.1 Load data from Object Storage

To get the data and load it into a pandas DataFrame:

1. Go to the [IBM Watson Facebook posts for 2015](https://apsportal.ibm.com/exchange/public/entry/view/ebaf6aab3f42217a22fb787d296fde9d) data card and click the download icon to save the file on your computer.
1. Back in your notebook, load the file by clicking the **Find and Add Data** icon and then dragging and dropping the file onto the pane or browsing for the file. The data is stored in the object storage container that is associated with your project.
1. Click in the next cell and then choose **Insert to code > Insert pandas DataFrame** from below the file name and then run the cell. Change the inserted variable name to `df_data_1`

Note: There is an issue (https://github.com/IBM/pixiedust-facebook-analysis/issues/39)
 that causes failure of non utf-8 encodings that requires a workaround.
 You would fix this in the cell below by adding an encoding parameter to read_csv(). For our example_facebook_data.csv:

  `df_data_1 = pd.read_csv(body, encoding='latin-1')` 
###  <span style="color: red"> _User Input_</span> 

In [22]:
# **Insert to code > Insert pandas DataFrame**
#df_data_1 = pd.read_csv('C:\\Users\\Rafael\\Downloads\\us-consumer-finance-complaint-database\\consumer_complaints_only_with_narrative_100.csv', encoding='latin-1')#, low_memory=False)
df_data_1 = pd.read_csv('C:\\Users\\Rafael\\Downloads\\us-consumer-finance-complaint-database\\consumer_complaints_only_with_narrative_100.csv', sep=',', error_bad_lines=False, index_col=False, dtype="unicode")

### <a id='load2'></a>2.2 Set variables
You need to set these variables:
 - The name of the DataFrame
 - Your credentials for the source file
 - A file name for the enriched DataFrame
 
Define a variable, `df`, for the DataFrame that you just created. If necessary, change the original DataFrame name to match the one you created.

In [24]:
# Make sure this uses the variable above. The number will vary in the inserted code.
try:
    df = df_data_1
    print(df.head())
except NameError as e:
    print('Error: Setup is incorrect or incomplete.\n')
    print('Follow the instructions to insert the pandas DataFrame above, and edit to')
    print('make the generated df_data_# variable match the variable used here.')
    raise

  date_received          product                            sub_product  \
0     3/19/2015  Debt collection  Other (i.e. phone, health club, etc.)   
1     3/19/2015    Consumer Loan                           Vehicle loan   
2     3/19/2015         Mortgage            Conventional fixed mortgage   
3     3/19/2015         Mortgage            Conventional fixed mortgage   
4     3/19/2015         Mortgage            Conventional fixed mortgage   

                                      issue      sub_issue  \
0     Cont'd attempts collect debt not owed  Debt was paid   
1                Managing the loan or lease            NaN   
2  Loan modification,collection,foreclosure            NaN   
3  Loan servicing, payments, escrow account            NaN   
4            Credit decision / Underwriting            NaN   

                        consumer_complaint_narrative company_public_response  \
0  XXXX has claimed I owe them {$27.00} for XXXX ...                     NaN   
1  Due to incons

**Select the cell below and place your cursor on an empty line below the comment.** 
Put in the credentials for the file you want to enrich by clicking on the 10/01 (upper right), then click `Insert to code` under the file you want to enrich, and choose `Insert Credentials`.

**Change the inserted variable name to `credentials_1`**

###  <span style="color: red"> _User Input_</span> 

In [6]:
# insert credentials for file - Change to credentials_1


In [7]:
# Make sure this uses the variable above. The number will vary in the inserted code.
try:
    credentials = credentials_1
except NameError as e:
    print('Error: Setup is incorrect or incomplete.\n')
    print('Follow the instructions to insert the file credentials above, and edit to')
    print('make the generated credentials_# variable match the variable used here.')
    raise

Error: Setup is incorrect or incomplete.

Follow the instructions to insert the file credentials above, and edit to
make the generated credentials_# variable match the variable used here.


NameError: name 'credentials_1' is not defined

<a id='prepare'></a>
## 3. Prepare Data
You'll prepare the data by cleansing it and extracting the URLs. Many of the posts contain both text and a URL. The first task is to separate URLs from the text so that they can be analyzed separately. Then you need to get thumbnails for the photos and links, and convert any shortened URLs to full URLs.

<a id='prepare1'></a>
###  3.1 Data Cleansing with Python
Renaming columns, removing noticeable noise in the data, pulling out URLs and appending to a new column to run through NLU.

To cleanse the data, you'll rename a column and add a column with the URLs that were embedded in the post. 

Change the name of the `Post Message` column to `Text`:

In [8]:
df.rename(columns={'Post Message': 'consumer_complaint_narrative'}, inplace=True)

In [9]:
# Drop the rows that have NaN for the text.
df.dropna(subset=['consumer_complaint_narrative'], inplace=True)

Use the `str.partition` function to remove strings that contain "http" and "www" from the `Text` column and save them in new DataFrames, then add all web addresses to a new `Link` column in the original DataFrame. This process captures all web addresses: https, http, and www.

In [10]:
df_http = df["consumer_complaint_narrative"].str.partition("http")
df_www = df["consumer_complaint_narrative"].str.partition("www")

# Combine delimiters with actual links
df_http["Link"] = df_http[1].map(str) + df_http[2]
df_www["Link1"] = df_www[1].map(str) + df_www[2]

# Include only Link columns
df_http.drop(df_http.columns[0:3], axis=1, inplace = True)
df_www.drop(df_www.columns[0:3], axis=1, inplace = True)

# Merge http and www DataFrames
dfmerge = pd.concat([df_http, df_www], axis=1)

# The following steps will allow you to merge data columns from the left to the right
dfmerge = dfmerge.apply(lambda x: x.str.strip()).replace('', np.nan)

# Use fillna to fill any blanks with the Link1 column
dfmerge["Link"].fillna(dfmerge["Link1"], inplace = True)

# Delete Link1 (www column)
dfmerge.drop("Link1", axis=1, inplace = True)

# Combine Link data frame
df = pd.concat([dfmerge,df], axis = 1)

# Make sure text column is a string
df["Text"] = df["consumer_complaint_narrative"].astype("str")

# Strip links from Text column
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(lambda x: x.split('http')[0])
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(lambda x: x.split('www')[0])

### <a id='prepare2'></a> 3.2 Extract thumbnails and extended links

A standard Facebook export does not provide the thumbnail that usually summarizes the link or photo associated with each post. Use the Beautiful Soup library to go into the HTML of the post and extract the thumbnail text:

In [ ]:
# Change links from objects to strings
for link in df.Link:
    df.Link.to_string()

piclinks = []
description = []
for url in df["Link"]:
    if pd.isnull(url):
        piclinks.append("")
        description.append("")
        continue
        
    try:
        # Skip certificate check with verify=False.
        # Don't do this if your urls are not secure.
        page3 = requests.get(url, verify=False)
        if page3.status_code != requests.codes.ok:
            piclinks.append("")
            description.append("")
            continue
    except Exception as e:
        print("Skipping url %s: %s" % (url, e))
        piclinks.append("")
        description.append("")
        continue
        
    soup3 = bs(page3.text,"lxml")
    
    pic = soup3.find('meta', property ="og:image")
    if pic:
        piclinks.append(pic["content"])
    else: 
        piclinks.append("")
    
    content = None
    desc = soup3.find(attrs={'name':'Description'})
    if desc:
        content = desc['content']
    if not content or content == 'null':
        # Try again with lowercase description
        desc = soup3.find(attrs={'name':'description'})
        if desc:
            content = desc['content']
    if not content or content == 'null':
        description.append("")
    else:
        description.append(content)
            
# Save thumbnail descriptions to df in a column titled 'Thumbnails'
df["Thumbnails"] = description
# Save image links to df in a column titled 'Image'
df["Image"] = piclinks

Convert shortened links to full links.
Use requests module to pull extended links. This is only necessary if the Facebook page uses different links than the articles themselves. For this example we are using IBM Watson's Facebook export which uses an IBM link. 


In [ ]:
shortlink = df["Link"]
extendedlink = []

for link in shortlink:
    if isinstance(link, float):  # Float is not a URL, probably NaN.
        extendedlink.append('')
    else:
        try:
            extended_link = requests.Session().head(link, allow_redirects=True).url
            extendedlink.append(extended_link)
        except Exception as e:
            print("Skipping link %s: %s" % (link, e))
            extendedlink.append('')

df["Extended Links"] = extendedlink

<a id='enrich'></a> 
## 4. Enrichment Time!
<a id='nlupost'></a>
###  4.1 NLU for the Post Text
The following script is an example of how to use Natural Language Understanding to iterate through each post and extract enrichment features for future analysis.

For this example, we are looking at the `Text` column in our DataFrame, which contains the text of each post. NLU can also iterate through a column of URLs, or other freeform text. There's a list within a list for the Keywords and Entities features to allow gathering multiple entities and keywords from each piece of text.

Each extracted feature is appended to the DataFrame in a new column that's defined at the end of the script. If you want to run this same script for the other columns, set the loop iterable to the column name, if you are using URLs, change the `text=response` parameter to `url=response`, and update the new column names as necessary. 

In [28]:
# Define the list of features to get enrichment values for entities, keywords, emotion and sentiment
#features = Features(entities=EntitiesOptions(), keywords=KeywordsOptions(), emotion=EmotionOptions(), sentiment=SentimentOptions())

features = Features(sentiment=SentimentOptions())

overallSentimentScore = []
overallSentimentType = []
highestEmotion = []
highestEmotionScore = []
kywords = []
entities = []

# Go through every response and enrich the text using NLU.
for text in df['consumer_complaint_narrative']:
    print(text)
    # We are assuming English to avoid errors when the language cannot be detected.
    enriched_json = nlu.analyze(text=text, features=features, language='en').get_result()
    print(json.dumps(enriched_json))

    # Get the SENTIMENT score and type
    if 'sentiment' in enriched_json:
        if('score' in enriched_json['sentiment']["document"]):
            overallSentimentScore.append(enriched_json["sentiment"]["document"]["score"])
        else:
            overallSentimentScore.append('0')

        if('label' in enriched_json['sentiment']["document"]):
            overallSentimentType.append(enriched_json["sentiment"]["document"]["label"])
        else:
            overallSentimentType.append('0')
    else:
        overallSentimentScore.append('0')
        overallSentimentType.append('0')

    # Read the EMOTIONS into a dict and get the key (emotion) with maximum value
    if 'emotion' in enriched_json:
        me = max(enriched_json["emotion"]["document"]["emotion"].items(), key=operator.itemgetter(1))[0]
        highestEmotion.append(me)
        highestEmotionScore.append(enriched_json["emotion"]["document"]["emotion"][me])
    else:
        highestEmotion.append("")
        highestEmotionScore.append("")

    # Iterate and get KEYWORDS with a confidence of over 70%
    if 'keywords' in enriched_json:
        tmpkw = []
        for kw in enriched_json['keywords']:
            if(float(kw["relevance"]) >= 0.7):
                tmpkw.append(kw["text"])
        # Convert multiple keywords in a list to a string and append the string
        kywords.append(', '.join(tmpkw))
    else:
        kywords.append("")
            
    # Iterate and get Entities with a confidence of over 30%
    if 'entities' in enriched_json:
        tmpent = []
        for ent in enriched_json['entities']: 
            if(float(ent["relevance"]) >= 0.3):
                tmpent.append(ent["type"])
 
        # Convert multiple entities in a list to a string and append the string
        entities.append(', '.join(tmpent))
    else:
        entities.append("")    
    
# Create columns from the list and append to the DataFrame
if highestEmotion:
    df['TextHighestEmotion'] = highestEmotion
if highestEmotionScore:
    df['TextHighestEmotionScore'] = highestEmotionScore

if overallSentimentType:
    df['TextOverallSentimentType'] = overallSentimentType
if overallSentimentScore:
    df['TextOverallSentimentScore'] = overallSentimentScore

df['TextKeywords'] = kywords
df['TextEntities'] = entities

XXXX has claimed I owe them {$27.00} for XXXX years despite the PROOF of PAYMENT I sent them : canceled check and their ownPAID INVOICE for {$27.00}! 
They continue to insist I owe them and collection agencies are after me. 
How can I stop this harassment for a bill I already paid four years ago? 

{"usage": {"text_characters": 299, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.696952}}, "language": "en"}
Due to inconsistencies in the amount owed that I was told by M & T Bank and the amount that was reported to the credit reporting agencies, I was advised to write a good will letter in order to address the issue and request the negative entry be removed from my credit report all together. I had a vehicle that was stolen and it was declared a total loss by insurance company. The insurance company and the GAP insurancw companypaid the outstanding balance of the loan, but I was told by M & T Bank that there was still a balance due on the loan

{"usage": {"text_characters": 409, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "positive", "score": 0.560555}}, "language": "en"}
This complaint is against Wells Fargo Bank for violations of the terms and conditions of Federal government 's mortgage modification program ( HARP ). Violations of the law include but are not limited to the following:1 ) A blatant miscalculation of income. ( Adding a 25 % increase in income on a household contributor even when it was not supported by fact, law, or the documents submitted ) 2 ) Taking longer than XXXX days to deny the appeal. 
3 ) Not following the guidelines for the application and appeal process set forth by the Federal government for loan modifications. 
4 ) Wells Fargo falsely falsely accused customer of not submitting enough documentation to support appeal of original denial. 
5 ) Customer we given false and conflicting reasons for denial. 

{"usage": {"text_characters": 773, "text_units": 1, "features": 1}, "se

{"usage": {"text_characters": 3037, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.243825}}, "language": "en"}
I am very disappointed that the CFPB did not help to resolve this fraudulant loan. # XXXX between XXXX XXXX and XXXX, FF ( AKA ) One West. Case # XXXX with CFPB. 

{"usage": {"text_characters": 163, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.20672}}, "language": "en"}
Mel S. Harris & assoc. used deceptive collection practices while attempting to collect on a purchased debt. Once initial contact had been made and a discussion on solutions to resolve the debt, the attorney office sent out a summons for an appearance in court. The representative had advised us that " we should not worry because we will receive several of these notifications. '' However, there was a case filed in the city court and required an immediate response to avoid a default judgment. This summons was used as a co

{"usage": {"text_characters": 447, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.350514}}, "language": "en"}
Nationstar Mortgage withheld money from my monthly house payment to fund an Escrow account from which they are to pay property taxes. They used the funds from my account to pay taxes on someone else 's property - landing me in collections with the XXXX County tax department. They have since paid off the collections, but are showing the additional amount as being part of my payoff amount ( meaning they are charging me XXXX for property taxes ). When my attorney and I have requested that they update the payoff since it was not my error, they are not helping me. They have told me they have to wait for the refund from XXXX County. When I contacted XXXX County, they advised me they do not refund mortgage companies for issues such as this. They said the mortgage company has to request the funds from the owner of the property that they err

{"usage": {"text_characters": 2815, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.478977}}, "language": "en"}
I have been disputing incorrect information on my Experian XXXX XXXX XXXX credit report but they are unwilling to allow me to dispute their " investigation results ''. The creditors are XXXX XXXX and XXXX XXXX XXXX. The report reflects late payments and the information is inaccurate. XXXX creditors have agreed to have the items corrected on all XXXX credit bureaus but only XXXX made the correction. 

{"usage": {"text_characters": 403, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.82584}}, "language": "en"}
I have been disputing incorrect information on my XXXX & Trans Union credit report but they are unwilling to allow me to dispute their " investigation results ''. The creditors are XXXX XXXX and XXXX XXXX XXXX. The report reflects late payments and the information is inaccurate. Both

...

{"usage": {"text_characters": 3161, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.542914}}, "language": "en"}
Ocwen loan servicing is the servicer of my loan. 
1. They refuse to talk to about anything. 
2. They refuse to send me any papers. 
3. They will not send me my statements4. They will not send payment statement. 
5. They will not return any letters or phone calls. 
6. They refuse to let me know what owed on loan. 
7. They refuse to let me know if they received my payments. 
8. They refuse to let me know what my payments are. 
9. They but forced insurance on this property when I have insurance on the property. They refused to take it off after 3 months of sending them proof and calling them over XXXX times. 
10. They double charged me for insurance. 
11. The only thing that they will say when I call, is that they were told not to talk or send anything to me due to I filed litigation against them. 
XXXX. I have over XXXX documented ca

{"usage": {"text_characters": 3294, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "positive", "score": 0.0458037}}, "language": "en"}
This company continues to report on my credit report after I sent them a letter telling them that this account was not mine and I have no idea what it is or who it belongs to! 

I asked for proof of a signed contract, I asked for a license to collect in my state, I asked for copies of all information referenced for this debt and still to date, I have not received anything but harassment from this company! 

THIS IS NOT MY DEBT! 

I WANT THIS ACCOUNT REMOVED FROM MY CREDIT REPORT AND THIS COMPANY TO STOP CONTACTING ME IMMEDIATELY! 

{"usage": {"text_characters": 537, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.650529}}, "language": "en"}
I received a court XXXX for a XXXX bill from XXXX years ago from a debt collection agency. I called immediately to ask what it 's from and they would

{"usage": {"text_characters": 1828, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.307655}}, "language": "en"}
SOMEONE USE MY INFORMATION. THIS ISNT MY ACCOUNT. 

{"usage": {"text_characters": 51, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.867716}}, "language": "en"}
I sent a letter however, I do no reply. Unfortunately, I was the victim of identity theft. I have done everything required of me by law to dispute the fraudulent items on my credit report. I sent a letter to " Tide Water Finance Company '' on XXXX via XXXX certified mail. The company refuses to honor my rights under the FDCPA & the FCRA as a victim of identity theft to remove the fraudulent inquiry that was submitted without my knowledge, consent and/or authorization. 

Please find attached the notice that was sent out along with the attached certified receipt. Information can be tracked online using referenced information. Than

{"usage": {"text_characters": 3818, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.403876}}, "language": "en"}
I have been approached by Concord Resolution ( & XXXX ; XXXXXXXXXXXX & gt ; : advising me that I had a payday loan from XXXX 2013. I asked for a signed document indicating my agreement. " XXXX XXXX '' sent me a collection letter from his organization from another collection agency by the name of XXXX XXXX XXXX. 
I have taken out payday loans but I can assuredly state I did n't get XXXX at that time. 

{"usage": {"text_characters": 404, "text_units": 1, "features": 1}, "sentiment": {"document": {"label": "negative", "score": -0.683177}}, "language": "en"}
I have been disputing with Experian for over 1.5 years for inaccurate reports and fraudulent reports and XXXX loans that was deleted then put back on and I have fraud alert with security freeze with my number some one changed my fraud alert phone number and put there number in ther

WatsonApiException: Error: invalid request, Code: 400 , X-dp-watson-tran-id: gateway01-1030510727 , X-global-transaction-id: 7ecac92c5bbe796d3d6c5887

After we extract all of the Keywords and Entities from each Post, we have columns with multiple Keywords and Entities separated by commas. For our Analysis in Part II, we also wanted the top Keyword and Entity for each Post. Because of this, we added two new columns to capture the `MaxTextKeyword` and `MaxTextEntity`.

In [26]:
df.head()
print(df)

# Choose first of Keywords,Concepts, Entities
df["MaxTextKeywords"] = df["TextKeywords"].apply(lambda x: x.split(',')[0])
df["MaxTextEntity"] = df["TextEntities"].apply(lambda x: x.split(',')[0])

      date_received           product                            sub_product  \
0         3/19/2015   Debt collection  Other (i.e. phone, health club, etc.)   
1         3/19/2015     Consumer Loan                           Vehicle loan   
2         3/19/2015          Mortgage            Conventional fixed mortgage   
3         3/19/2015          Mortgage            Conventional fixed mortgage   
4         3/19/2015          Mortgage            Conventional fixed mortgage   
5         3/19/2015          Mortgage            Conventional fixed mortgage   
6         3/19/2015          Mortgage            Conventional fixed mortgage   
7         3/19/2015          Mortgage            Conventional fixed mortgage   
8         3/19/2015       Credit card                                    NaN   
9         3/19/2015     Consumer Loan                           Vehicle loan   
10        3/19/2015          Mortgage                       Reverse mortgage   
11        3/19/2015       Credit card   

KeyError: 'TextKeywords'

<a id='nlutn'></a>
###  4.2 NLU for Thumbnail Text

We will repeat the same process for Thumbnails and Article Text.

In [ ]:
# Define the list of features to get enrichment values for entities, keywords, emotion and sentiment
features = Features(entities=EntitiesOptions(), keywords=KeywordsOptions(), emotion=EmotionOptions(), sentiment=SentimentOptions())

overallSentimentScore = []
overallSentimentType = []
highestEmotion = []
highestEmotionScore = []
kywords = []
entities = []

# Go through every response and enrich the text using NLU.
for text in df['Thumbnails']:
    if not text:
        overallSentimentScore.append(' ')
        overallSentimentType.append(' ')
        highestEmotion.append(' ')
        highestEmotionScore.append(' ')
        kywords.append(' ')
        entities.append(' ')
        continue

    enriched_json = nlu.analyze(text=text, features=features, language='en').get_result()

    # Get the SENTIMENT score and type
    if 'sentiment' in enriched_json:
        if('score' in enriched_json['sentiment']["document"]):
            overallSentimentScore.append(enriched_json["sentiment"]["document"]["score"])
        else:
            overallSentimentScore.append("")

        if('label' in enriched_json['sentiment']["document"]):
            overallSentimentType.append(enriched_json["sentiment"]["document"]["label"])
        else:
            overallSentimentType.append("")

    # Read the EMOTIONS into a dict and get the key (emotion) with maximum value
    if 'emotion' in enriched_json:
        me = max(enriched_json["emotion"]["document"]["emotion"].items(), key=operator.itemgetter(1))[0]
        highestEmotion.append(me)
        highestEmotionScore.append(enriched_json["emotion"]["document"]["emotion"][me])

    else:
        highestEmotion.append("")
        highestEmotionScore.append("")

    # Iterate and get KEYWORDS with a confidence of over 70%
    if 'keywords' in enriched_json:
        tmpkw = []
        for kw in enriched_json['keywords']:
            if(float(kw["relevance"]) >= 0.7):
                tmpkw.append(kw["text"])
        # Convert multiple keywords in a list to a string and append the string
        kywords.append(', '.join(tmpkw))
     
    # Iterate and get Entities with a confidence of over 30%
    if 'entities' in enriched_json:
        tmpent = []
        for ent in enriched_json['entities']:              
            if(float(ent["relevance"]) >= 0.3):
                tmpent.append(ent["type"])
        # Convert multiple entities in a list to a string and append the string
        entities.append(', '.join(tmpent))
    else:
        entities.append("")     
  
# Create columns from the list and append to the DataFrame
if highestEmotion:
    df['ThumbnailHighestEmotion'] = highestEmotion
if highestEmotionScore:
    df['ThumbnailHighestEmotionScore'] = highestEmotionScore

if overallSentimentType:
    df['ThumbnailOverallSentimentType'] = overallSentimentType
if overallSentimentScore:
    df['ThumbnailOverallSentimentScore'] = overallSentimentScore

df['ThumbnailKeywords'] = kywords
df['ThumbnailEntities'] = entities

 Add two new columns to capture the `MaxThumbnailKeyword` and `MaxThumbnailEntity`:

In [ ]:
# Set 'Max' to first one from keywords and entities lists
df["MaxThumbnailKeywords"] = df["ThumbnailKeywords"].apply(lambda x: x.split(',')[0])
df["MaxThumbnailEntity"] = df["ThumbnailEntities"].apply(lambda x: x.split(',')[0])

<a id='nlulink'></a> 
### 4.3 NLU for Article Text

In [ ]:
# Define the list of features to get enrichment values for entities, keywords, emotion and sentiment
features = Features(entities=EntitiesOptions(), keywords=KeywordsOptions(), emotion=EmotionOptions(), sentiment=SentimentOptions())

overallSentimentScore = []
overallSentimentType = []
highestEmotion = []
highestEmotionScore = []
kywords = []
entities = []
article_text = []
        
# Go through every response and enrich the article using NLU
for url in df['Extended Links']:
    if not url:
        overallSentimentScore.append(' ')
        overallSentimentType.append(' ')
        highestEmotion.append(' ')
        highestEmotionScore.append(' ')
        kywords.append(' ')
        entities.append(' ')
        article_text.append(' ')
        continue

    # Run links through NLU to get entities, keywords, emotion and sentiment.
    # Use return_analyzed_text to extract text for Tone Analyzer to use.
    
    try:
        enriched_json = nlu.analyze(url=url,
                                features=features,
                                language='en',
                                return_analyzed_text=True).get_result()
        article_text.append(enriched_json["analyzed_text"])
    except Exception as e:
        print("Skipping url %s: %s" % (url, e))
        overallSentimentScore.append(' ')
        overallSentimentType.append(' ')
        highestEmotion.append(' ')
        highestEmotionScore.append(' ')
        kywords.append(' ')
        entities.append(' ')
        article_text.append(' ')
        continue
      
    
    

    # Get the SENTIMENT score and type
    if 'sentiment' in enriched_json:
        if('score' in enriched_json['sentiment']["document"]):
            overallSentimentScore.append(enriched_json["sentiment"]["document"]["score"])
        else:
            overallSentimentScore.append('None')

        if('label' in enriched_json['sentiment']["document"]):
            overallSentimentType.append(enriched_json["sentiment"]["document"]["label"])
        else:
            overallSentimentType.append('')

    # Read the EMOTIONS into a dict and get the key (emotion) with maximum value
    if 'emotion' in enriched_json:
        me = max(enriched_json["emotion"]["document"]["emotion"].items(), key=operator.itemgetter(1))[0]
        highestEmotion.append(me)
        highestEmotionScore.append(enriched_json["emotion"]["document"]["emotion"][me])

    else:
        highestEmotion.append('')
        highestEmotionScore.append('')

    # Iterate and get KEYWORDS with a confidence of over 70%
    if 'keywords' in enriched_json:
        tmpkw = []
        for kw in enriched_json['keywords']:
            if(float(kw["relevance"]) >= 0.7):
                tmpkw.append(kw["text"])
        # Convert multiple keywords in a list to a string and append the string
        kywords.append(', '.join(tmpkw))
    else: 
        kywords.append("")
            
    # Iterate and get Entities with a confidence of over 30%
    if 'entities' in enriched_json:
        tmpent = []
        for ent in enriched_json['entities']:               
            if(float(ent["relevance"]) >= 0.3):
                tmpent.append(ent["type"])
        # Convert multiple entities in a list to a string and append the string
        entities.append(', '.join(tmpent))
    else:
        entities.append("")
    
# Create columns from the list and append to the DataFrame
if highestEmotion:
    df['LinkHighestEmotion'] = highestEmotion
if highestEmotionScore:
    df['LinkHighestEmotionScore'] = highestEmotionScore

if overallSentimentType:
    df['LinkOverallSentimentType'] = overallSentimentType
if overallSentimentScore:
    df['LinkOverallSentimentScore'] = overallSentimentScore

df['LinkKeywords'] = kywords
df['LinkEntities'] = entities
df['Article Text'] = article_text

Add two new columns to capture the `MaxLinkKeyword` and `MaxLinkEntity`:

In [ ]:
# Set 'Max' to first one from keywords and entities lists
df["MaxLinkKeywords"] = df["LinkKeywords"].apply(lambda x: x.split(',')[0])
df["MaxLinkEntity"] = df["LinkEntities"].apply(lambda x: x.split(',')[0])

### <a id='tonepost'></a>4.4 Tone Analyzer for Post Text
Now use the Tone Analyzer to gather the top social, writing, and emotion tones from the post text and append them, along with their respective scores, to the DataFrame. 

In [ ]:
highestEmotionTone = []
emotionToneScore = []

highestLanguageTone = []
languageToneScore = []

highestSocialTone = []
socialToneScore = []

for text in df['Text']:
    enriched_json = tone_analyzer.tone(text, content_type='text/plain').get_result()
        
    me = max(enriched_json["document_tone"]["tone_categories"][0]["tones"], key = itemgetter('score'))['tone_name']      
    highestEmotionTone.append(me)
    you = max(enriched_json["document_tone"]["tone_categories"][0]["tones"], key = itemgetter('score'))['score']
    emotionToneScore.append(you)
            
    me1 = max(enriched_json["document_tone"]["tone_categories"][1]["tones"], key = itemgetter('score'))['tone_name']      
    highestLanguageTone.append(me1)
    you1 = max(enriched_json["document_tone"]["tone_categories"][1]["tones"], key = itemgetter('score'))['score']
    languageToneScore.append(you1)
            
    me2 = max(enriched_json["document_tone"]["tone_categories"][2]["tones"], key = itemgetter('score'))['tone_name']      
    highestSocialTone.append(me2)
    you2 = max(enriched_json["document_tone"]["tone_categories"][2]["tones"], key = itemgetter('score'))['score']
    socialToneScore.append(you2)
    
df['highestEmotionTone'] = highestEmotionTone    
df['emotionToneScore'] = emotionToneScore
df['languageToneScore'] = languageToneScore
df['highestLanguageTone'] = highestLanguageTone
df['highestSocialTone'] = highestSocialTone    
df['socialToneScore'] = socialToneScore

<a id='tonearticle'></a> 
### 4.5 Tone Analyzer for Article Text

Unlike NLU, Tone Analyzer cannot iterate through a URL, so we used NLU (above) to pull the Article Text from the URL and append it to the DataFrame.

Now using Tone Analyzer, we iterate through the `Article Text` column which contains all of the free form text from the articles contained in the Facebook posts. 

We are using Tone Analyzer to gather the top Social, Language and Emotion Tones from the Articles.

In [ ]:
highestEmotionTone = []
emotionToneScore = []

highestLanguageTone = []
languageToneScore = []

highestSocialTone = []
socialToneScore = []

for text in df['Article Text']:
    if not text:
        emotionToneScore.append(' ')
        highestEmotionTone.append(' ')     
        languageToneScore.append(' ')
        highestLanguageTone.append(' ')     
        socialToneScore.append(' ')
        highestSocialTone.append(' ')  
        continue

    enriched_json = tone_analyzer.tone(text, content_type='text/plain', sentences=False).get_result()
        
    maxTone = max(enriched_json["document_tone"]["tone_categories"][0]["tones"], key = itemgetter('score'))['tone_name']      
    highestEmotionTone.append(maxTone)
    maxToneScore = max(enriched_json["document_tone"]["tone_categories"][0]["tones"], key = itemgetter('score'))['score']
    emotionToneScore.append(maxToneScore)
            
    maxLanguageTone = max(enriched_json["document_tone"]["tone_categories"][1]["tones"], key = itemgetter('score'))['tone_name']      
    highestLanguageTone.append(maxLanguageTone)
    maxLanguageScore = max(enriched_json["document_tone"]["tone_categories"][1]["tones"], key = itemgetter('score'))['score']
    languageToneScore.append(maxLanguageScore)
            
    maxSocial = max(enriched_json["document_tone"]["tone_categories"][2]["tones"], key = itemgetter('score'))['tone_name']      
    highestSocialTone.append(maxSocial)
    maxSocialScore = max(enriched_json["document_tone"]["tone_categories"][2]["tones"], key = itemgetter('score'))['score']
    socialToneScore.append(maxSocialScore)

df['articlehighestEmotionTone'] = highestEmotionTone
df['articleEmotionToneScore'] = emotionToneScore
df['articlelanguageToneScore'] = languageToneScore
df['articlehighestLanguageTone'] = highestLanguageTone
df['articlehighestSocialTone'] = highestSocialTone
df['articlesocialToneScore'] = socialToneScore

<a id='visual'></a> 
### 4.6 Visual Recognition
Below uses Visual Recognition to classify the thumbnail images.

> NOTE: When using the **free tier** of Visual Recognition, _classify_ has a limit of 250 images per day.

In [ ]:
piclinks = df["Image"]

picclass = []
piccolor = []
pictype1 = []
pictype2 = []
pictype3 = []

for pic in piclinks:
    if not pic or pic == 'default-img':
        picclass.append(' ')
        piccolor.append(' ')
        pictype1.append(' ')
        pictype2.append(' ')
        pictype3.append(' ')
        continue

    classes = []
    enriched_json = {}
    try:
        enriched_json = visual_recognition.classify(url=pic).get_result()
    except Exception as e:
        print("Skipping url %s: %s" % (pic, e))

    if 'error' in enriched_json:
        print(enriched_json['error'])
    if 'images' in enriched_json and 'classifiers' in enriched_json['images'][0]:
        classes = enriched_json['images'][0]["classifiers"][0]["classes"]

    color1 = None
    class1 = None
    type_hierarchy1 = None

    for iclass in classes:
        # Grab the first color, first class, and first type hierarchy.
        # Note: Usually you'd filter by 'score' too.
        if not type_hierarchy1 and 'type_hierarchy' in iclass:
            type_hierarchy1 = iclass['type_hierarchy']
        if not class1:
            class1 = iclass['class']
        if not color1 and iclass['class'].endswith(' color'):
            color1 = iclass['class'][:-len(' color')]
        if type_hierarchy1 and class1 and color1:
            # We are only using 1 of each per image. When we have all 3, break.
            break

    picclass.append(class1 or ' ')
    piccolor.append(color1 or ' ')
    type_split = (type_hierarchy1 or '/ / / ').split('/')
    pictype1.append(type_split[1] if len(type_split) > 1 else '-')
    pictype2.append(type_split[2] if len(type_split) > 2 else '- ')
    pictype3.append(type_split[3] if len(type_split) > 3 else '-')

df["Image Color"] = piccolor
df["Image Class"] = picclass
df["Image Type"] = pictype1
df["Image Subtype"] = pictype2
df["Image Subtype2"] = pictype3

 <a id='write'></a>
## Enrichment is now COMPLETE!
<a id='write1'></a> 
Save a copy of the enriched DataFrame as a file in Cloud Object Storage. To run the upload_file function we first need to create a variable that contains our credentials we created in section 2.2. No user input is required as we already have all of the information we need. To upload the file to COS simply run the next two cells.

In [ ]:
cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [ ]:
# Build the enriched file name from the original filename.
localfilename = 'enriched_' + credentials['FILE']

# Write a CSV file from the enriched pandas DataFrame.
df.to_csv(localfilename, index=False)

# Use the above put_file method with credentials to put the file in Object Storage.
cos.upload_file(localfilename, Bucket=credentials['BUCKET'],Key=localfilename)

<a id="part2"></a> 
# Part II - Data Preparation
<a id='prepare'></a>
## 1. Prepare Data
 <a id='visualizations'></a>
### 1.1 Prepare Multiple DataFrames for Visualizations
Before we can create the separate tables for each Watson feature we need to organize and reformat the data. First, we need to determine which data points are tied to metrics. Second, we need to make sure make sure each metric is numeric. _(This is necessary for PixieDust in Part III)_

In [ ]:
# Determine which data points are tied to metrics and put them in a list
metrics = ["Lifetime Post Total Reach", "Lifetime Post organic reach", "Lifetime Post Paid Reach", "Lifetime Post Total Impressions", "Lifetime Post Organic Impressions", 
           "Lifetime Post Paid Impressions", "Lifetime Engaged Users", "Lifetime Post Consumers", "Lifetime Post Consumptions", "Lifetime Negative feedback", "Lifetime Negative Feedback from Users", 
           "Lifetime Post Impressions by people who have liked your Page", "Lifetime Post reach by people who like your Page", "Lifetime Post Paid Impressions by people who have liked your Page", 
           "Lifetime Paid reach of a post by people who like your Page", "Lifetime People who have liked your Page and engaged with your post", "Lifetime Talking About This (Post) by action type - comment", 
           "Lifetime Talking About This (Post) by action type - like", "Lifetime Talking About This (Post) by action type - share", "Lifetime Post Stories by action type - comment", "Lifetime Post Stories by action type - like", 
           "Lifetime Post Stories by action type - share", "Lifetime Post consumers by type - link clicks", "Lifetime Post consumers by type - other clicks", "Lifetime Post consumers by type - photo view", "Lifetime Post Consumptions by type - link clicks", 
           "Lifetime Post Consumptions by type - other clicks", "Lifetime Post Consumptions by type - photo view", "Lifetime Negative feedback - hide_all_clicks", "Lifetime Negative feedback - hide_clicks", 
           "Lifetime Negative Feedback from Users by Type - hide_all_clicks", "Lifetime Negative Feedback from Users by Type - hide_clicks"]


<a id='tone'></a> 
### 1.2 Create a Consolidated Tone DataFrame
You'll create a DataFrame for the tones of the post text and a DataFrame for the tones of the article text. Then you'll combine them into one DataFrame.

Create a DataFrame for post tones:

#### Post Tone DataFrame

In [ ]:
# Create a list with only Post Tone Values
post_tones = ["Text","highestEmotionTone", "emotionToneScore", "languageToneScore", "highestLanguageTone", "highestSocialTone", "socialToneScore"]

# Append DataFrame with these metrics
post_tones.extend(metrics)

# Create a new DataFrame with tones and metrics
df_post_tones = df[post_tones]

# Determine which tone values are suppose to be numeric and ensure they are numeric. 
post_numeric_values = ["emotionToneScore", "languageToneScore", "socialToneScore"]
for i in post_numeric_values:
    df_post_tones[i] = pd.to_numeric(df_post_tones[i], errors='coerce')

# Make all metrics numeric
for i in metrics:
    df_post_tones[i] = pd.to_numeric(df_post_tones[i], errors='coerce')
    
# Drop NA Values in Tone Enrichment Columns
df_post_tones.dropna(subset=["socialToneScore"] , inplace = True)


# Add in a column to distinguish what portion the enrichment was happening 
df_post_tones["Type"] = "Post"

#### Article Tone DataFrame

In [ ]:
# Create a list with only Article Tone Values
article_tones = ["Text", "articlehighestEmotionTone", "articleEmotionToneScore", "articlelanguageToneScore", "articlehighestLanguageTone", "articlehighestSocialTone", "articlesocialToneScore"]

# Append DataFrame with these metrics
article_tones.extend(metrics)

# Create a new DataFrame with tones and metrics
df_article_tones = df[article_tones]

# Determine which values are suppose to be numeric and ensure they are numeric. 
art_numeric_values = ["articleEmotionToneScore", "articlelanguageToneScore", "articlesocialToneScore"]
for i in art_numeric_values:
    df_article_tones[i] = pd.to_numeric(df_article_tones[i], errors='coerce')
    
# Make all metrics numeric
for i in metrics:
    df_article_tones[i] = pd.to_numeric(df_article_tones[i], errors='coerce')
    
# Drop NA Values in Tone Enrichment Columns
df_article_tones.dropna(subset=["articlesocialToneScore"] , inplace = True)

# Add in a column to distinguish what portion the enrichment was happening 
df_article_tones["Type"] = "Article"

#### Combine Post and Article DataFrames to Make One Tone DataFrame

In [ ]:
# First make the Column Headers the same
df_post_tones.rename(columns={"highestEmotionTone":"Emotion Tone", "emotionToneScore":"Emotion Tone Score", "languageToneScore": "Language Tone Score", "highestLanguageTone": "Language Tone", "highestSocialTone": "Social Tone", "socialToneScore":"Social Tone Score"
}, inplace=True)

df_article_tones.rename(columns={"articlehighestEmotionTone":"Emotion Tone", "articleEmotionToneScore":"Emotion Tone Score", "articlelanguageToneScore": "Language Tone Score", "articlehighestLanguageTone": "Language Tone", "articlehighestSocialTone": "Social Tone", "articlesocialToneScore":"Social Tone Score"
}, inplace=True)

# Combine into one data frame
df_tones = pd.concat([df_post_tones, df_article_tones])

 <a id='keyword'></a> 
### 1.3 Create a Consolidated Keyword DataFrame
You'll create DataFrames for the keywords of the article text, the thumbnail text, and the post text. Then you'll combine them into one DataFrame.

 #### Article Keyword DataFrame 

In [ ]:
# Create a list with only Article Keywords
article_keywords = ["Text", "MaxLinkKeywords"]

# Append DataFrame with these metrics
article_keywords.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_article_keywords = df[article_keywords]

# Make all metrics numeric
for i in metrics:
    df_article_keywords[i] = pd.to_numeric(df_article_keywords[i], errors='coerce')

# Drop NA Values in Keywords Column

df_article_keywords['MaxLinkKeywords'].replace(' ', np.nan, inplace=True)
df_article_keywords.dropna(subset=['MaxLinkKeywords'], inplace=True)

# Add in a column to distinguish what portion the enrichment was happening 
df_article_keywords["Type"] = "Article"

#### Thumbnail Keyword DataFrame 

In [ ]:
# Create a list with only Thumbnail Keywords
thumbnail_keywords = ["Text", "MaxThumbnailKeywords"]

# Append DataFrame with these metrics
thumbnail_keywords.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_thumbnail_keywords = df[thumbnail_keywords]


# Make all metrics numeric
for i in metrics:
    df_thumbnail_keywords[i] = pd.to_numeric(df_thumbnail_keywords[i], errors='coerce')
    
# Drop NA Values in Keywords Column
df_thumbnail_keywords['MaxThumbnailKeywords'].replace(' ', np.nan, inplace=True)
df_thumbnail_keywords.dropna(subset=['MaxThumbnailKeywords'], inplace=True)

# Add in a column to distinguish what portion the enrichment was happening 
df_thumbnail_keywords["Type"] = "Thumbnails"

#### Post Keyword DataFrame 

In [ ]:
# Create a list with only Thumbnail Keywords
post_keywords = ["Text", "MaxTextKeywords"]

# Append DataFrame with these metrics
post_keywords.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_post_keywords = df[post_keywords]

# Make all metrics numeric
for i in metrics:
    df_post_keywords[i] = pd.to_numeric(df_post_keywords[i], errors='coerce')
    
# Drop NA Values in Keywords Column

df_post_keywords['MaxTextKeywords'].replace(' ', np.nan, inplace=True)
df_post_keywords.dropna(subset=['MaxTextKeywords'], inplace=True)

# Add in a column to distinguish what portion the enrichment was happening 
df_post_keywords["Type"] = "Posts"

#### Combine Post, Thumbnail, and Article DataFrames to Make One Keywords DataFrame

In [ ]:
# First make the column headers the same
df_post_keywords.rename(columns={"MaxTextKeywords": "Keywords"}, inplace=True)

df_thumbnail_keywords.rename(columns={"MaxThumbnailKeywords":"Keywords"}, inplace=True)

df_article_keywords.rename(columns={"MaxLinkKeywords":"Keywords"}, inplace=True)

# Combine into one data frame
df_keywords = pd.concat([df_post_keywords, df_thumbnail_keywords, df_article_keywords])

# Discard keywords with lower consumption to make charting easier
df_keywords = df_keywords[df_keywords["Lifetime Post Consumptions"] > 175]

<a id='entity'></a>
###  1.4 Create a Consolidated Entity DataFrame
You'll create DataFrames for the entities of the article text, the thumbnail text, and the post text. Then you'll combine them into one DataFrame.

#### Article Entity DataFrame 

In [ ]:
# Create a list with only Article Keywords
article_entities = ["Text", "MaxLinkEntity"]

# Append DataFrame with these metrics
article_entities.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_article_entities = df[article_entities]
    
# Make all metrics numeric
for i in metrics:
    df_article_entities[i] = pd.to_numeric(df_article_entities[i], errors='coerce')
    
# Drop NA Values in Keywords Column
df_article_entities['MaxLinkEntity'] = df["MaxLinkEntity"].replace(r'\s+', np.nan, regex=True)
df_article_entities.dropna(subset=['MaxLinkEntity'], inplace=True)

# Add in a column to distinguish what portion the enrichment was happening 
df_article_entities["Type"] = "Article"

#### Thumbnail Entity DataFrame

In [ ]:
# Create a list with only Thumbnail Keywords
thumbnail_entities = ["Text", "MaxThumbnailEntity"]

# Append DataFrame with these metrics
thumbnail_entities.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_thumbnail_entities = df[thumbnail_entities]

# Make all metrics numeric
for i in metrics:
    df_thumbnail_entities[i] = pd.to_numeric(df_thumbnail_entities[i], errors='coerce')
    
# Drop NA Values in Keywords Column
df_thumbnail_entities['MaxThumbnailEntity'] = df_thumbnail_entities['MaxThumbnailEntity'].replace(r'\s+', np.nan, regex=True)
df_thumbnail_entities.dropna(subset=['MaxThumbnailEntity'], inplace=True)

# Add in a column to distinguish what portion the enrichment was happening 
df_thumbnail_entities["Type"] = "Thumbnails"

#### Post Entity DataFrame

In [ ]:
# Create a list with only Thumbnail Keywords
post_entities = ["Text", "MaxTextEntity"]

# Append DataFrame with these metrics
post_entities.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_post_entities = df[post_entities]

# Make all metrics numeric
for i in metrics:
    df_post_entities[i] = pd.to_numeric(df_post_entities[i], errors='coerce')
    
# Drop NA Values in Keywords Column
df_post_entities['MaxTextEntity'] = df_post_entities['MaxTextEntity'].replace(r'\s+', np.nan, regex=True)
df_post_entities.dropna(subset=['MaxTextEntity'], inplace=True)

# Add in a column to distinguish what portion the enrichment was happening 
df_post_entities["Type"] = "Posts"

#### Combine Post, Thumbnail, and Article DataFrames to Make One Entity DataFrame

In [ ]:
# First make the column headers the same
df_post_entities.rename(columns={"MaxTextEntity": "Entities"}, inplace=True)

df_thumbnail_entities.rename(columns={"MaxThumbnailEntity":"Entities"}, inplace=True)

df_article_entities.rename(columns={"MaxLinkEntity":"Entities"}, inplace=True)

# Combine into one data frame
df_entities = pd.concat([df_post_entities, df_thumbnail_entities, df_article_entities])

df_entities["Entities"] = df_entities["Entities"].replace('', np.nan)
df_entities.dropna(subset=["Entities"], inplace=True)

<a id='image_dataframe'></a>
###  1.5 Create a Consolidated Image DataFrame

#### Combine Metrics with Type Hierarchy, Class and Color to Make One Image DataFrame

In [ ]:
# Create a list with only Visual Recognition columns
pic_keywords = ['Image Type', 'Image Subtype', 'Image Subtype2', 'Image Class', 'Image Color']

# Append DataFrame with these metrics
pic_keywords.extend(metrics)

# Create a new DataFrame with keywords and metrics
df_pic_keywords = df[pic_keywords]

# Make all metrics numeric
for i in metrics:
    df_pic_keywords[i] = pd.to_numeric(df_pic_keywords[i], errors='coerce')

<a id="part3"></a> 
# Part III
<a id='2setup'></a> 
## 1. Setup
<a id='2setup2'></a>
###  1.1 Assign Variables
Assign new DataFrames to variables. 

In [ ]:
entities = df_entities
tones = df_tones
keywords = df_keywords

<a id='2visual'></a>
##  2. Visualize Data
<a id='2visual1'></a> 
### 2.1 Run PixieDust Visualization Library with Display() API
PixieDust lets you visualize your data in just a few clicks using the display() API. You can find more info at https://ibm-cds-labs.github.io/pixiedust/displayapi.html.

#### We can use a pie chart to identify how post consumption was broken up by tone. 

Click on the `Options` button to change the chart.  Here are some things to try:
* Add *Type* to make the breakdown show *Post* or *Article*.
* Show *Social Tone* intead of *Emotion Tone* (or both).
* Try a different metric.

In [ ]:
display(tones)

#### Now let's look at the same statistics as a bar chart.

It is the same line of code. Use the `Edit Metadata` button to see how PixieDust knows to show us a bar chart. If you don't have a button use the menu and select `View > Cell Toolbar > Edit Metadata`.

A bar chart is better at showing more information. We added `Cluster By: Type` so we already see numbers for posts and articles. Notice what the chart tells you. *Joy* seems to be the dominant emotion. Click on `Options` and try this:

* Change the aggregation to `AVG`.

What emotion leads to higher average consumption?


In [ ]:
display(tones)

#### Now let's look at the entities that were detected by Natural Language Understanding.

The following bar chart shows the entities that were detected. This time we are stacking negative feedback and "likes" to get a picture of the kind of feedback the entities were getting. We chose a horizontal, stacked bar chart with descending values for a little variety.

* Try a different renderer and see what you get.

In [ ]:
display(entities)

#### Next we look at the keywords detected by Natural Language Understanding

This time we are using the bokeh renderer for a different look. The renderers have different capabilities.

In [ ]:
display(keywords)

#### Now let's take a look at what Visual Recognition can show us.

See how the images influenced the metrics. We've used visual recognition to identify a class and a type hierarchy for each image. We've also captured the top recognized color for each image. Our sample data doesn't have a significant number of data points, but these three charts demonstrate how you could:

1. Recognize image classes that correlate to higher average post consumption.
1. Add a type hierarchy for a higher level abstraction or to add grouping/stacking to the class data.
1. Determine if image color correlates to user reactions.

Visual recognition makes it surprisingly easy to do all of the above. Of course, you can easily try different metrics as you experiment. If you are not convinced that you should add ultramarine robot pictures to all of your articles, then you might want to do some research with a better data sample.

In [ ]:
display(df_pic_keywords)

In [ ]:
display(df_pic_keywords)

In [ ]:
display(df_pic_keywords)

# More Info.
For more information about PixieDust, check out the following:
* PixieDust Documentation: https://ibm-cds-labs.github.io/pixiedust/index.html
* PixieDust GitHub Repo: https://github.com/ibm-cds-labs/pixiedust

Visit the With Watson page to learn more about the work we are doing with our partners to make Watson and Watson Studio come to life for our clients
* With Watson Page: https://www.ibm.com/watson/with-watson/

Visit the Watson Accelerators portal to see more live patterns in action:
* Watson Accelerators: http://www.watsonaccelerators.com 


### About the Authors:
#### Tyler Andersen and Anna Quincy
Tyler and Anna are part of the With Watson team focused on pairing Cognitive Services with Data Science Patterns and Techniques for clients around the world. They work directly with clients to determine how they can use Watson and IBM's Watson Studio with their own data to solve a wide range of business issues.
 
#### Mark Sturdevant
Mark is a Developer Advocate at IBM where he implements open source code patterns to guide developers as they innovate using IBM's AI and data analytics solutions with Watson and Watson Studio.

<hr>
Copyright © IBM Corp. 2017, 2018. This notebook and its source code are released under the terms of the Apache 2.0.

Licensed under the Apache License, Version 2.0 (the "License"); you may
not use this file except in compliance with the License. You may obtain
a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
License for the specific language governing permissions and limitations
under the License.